## Imports

In [222]:
import pandas as pd
import numpy as np
import configparser
import os
import sys

In [223]:
cwd = os.getcwd()

In [224]:
module_path = os.path.join(cwd, '..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [225]:
import financialstatements

## Config

In [226]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\tasco\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Reading Data

In [227]:
TRIAL_BALANCE_OUTPUT = os.path.join(cwd, config.get("data_outputs_directory", "TRIAL_BALANCE_OUTPUT"))

In [228]:
COA_DATA = os.path.join(cwd, config.get('table_files', 'COA_DATA'))

In [229]:
tb_paths = financialstatements.creating_input_paths(TRIAL_BALANCE_OUTPUT)

In [230]:
tb_df = pd.read_csv(tb_paths[0])

In [231]:
coa_df = pd.read_excel(COA_DATA, sheet_name ='chart_of_accounts')

## ETL

In [232]:
tb_df = tb_df[['GL_Code', 'Account', 'Movement', 'End_Balance']]

In [233]:
tb_df = tb_df.rename(columns={'End_Balance' : 'Amount'})

In [234]:
coa_df = coa_df[['GL_Code', 'Category', 'Account_Type', 'Account_Classification']]

In [235]:
df = pd.merge(tb_df, coa_df, how = 'left', on = 'GL_Code')

In [236]:
df

,GL_Code,Account,Movement,Amount,Category,Account_Type,Account_Classification
0,100101,Free Checking Bank OZK,-2315.25,13857.66,Cash,Asset,Permanent
1,100102,Venmo Balance,195.23,195.23,Cash,Asset,Permanent
2,100202,Associate Stock Purchase Plan - Computer Share,161.00,2889.89,Short Term Investments,Asset,Permanent
3,100204,Cryptocurrency,40.00,586.96,Short Term Investments,Asset,Permanent
4,100301,Roth IRA - EdwardJones,4287.63,10702.95,Long Term Investments,Asset,Permanent
5,100304,Health Savings Account Investment,100.00,1734.47,Long Term Investments,Asset,Permanent
6,100305,Walmart Roth 401k - Merrill,549.24,10421.02,Long Term Investments,Asset,Permanent
7,200101,EdwardJones MasterCard,-1934.06,1934.06,Credit Card,Liability,Permanent
8,400101,Regular Earnings - Walmart Salary,-5492.32,5492.32,Income,Revenue,Temporary
9,400103,Walmart Stock Match,-21.00,21.00,Income,Revenue,Temporary


In [238]:
rev_df = df[df['Account_Type'] == 'Revenue']
total_rev = rev_df['Amount'].sum()

In [239]:
deductions_df = df[df['Account_Type'] == 'Deduction']
total_deductions = deductions_df['Amount'].sum()

In [240]:
rev_after_ded = total_rev - total_deductions